<a href="https://colab.research.google.com/github/GregSym/Numberphile_Follow_Alongs/blob/spoon/NumberphileFollowAlongs_StonesOnAnInfiniteChessboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stones on an Infinite Chessboard
https://www.youtube.com/watch?v=m4Uth-EaTZ8

A game in which a set of starting stones of value one are placed on a theoretically infinite grid in order to generate the largest possible sequence of natural numbers by evaluating the sum of a cell in the aforementioned grid's cells.

I suppose part of the challenge here is visualising that so it's not as confusing as I made it sound. The YT video is much easier to follow.

In [15]:
from __future__ import annotations
import collections
import numpy as np

## Principle setup for running the game

In [16]:
board: collections.defaultdict[tuple[int, int], int] = collections.defaultdict(lambda: 0)

In [17]:
def setup_init_state(board: collections.defaultdict[int], init_stones=2):
  if init_stones==2:
    board[(0, 0)] = 1
    board[(2, 2)] = 1

adjacency = ((-1, -1), (0, -1), (1, -1), (-1, 0), (0, 0), (1, 0), (0, 1), (1, 1))
assert len(set(adjacency)) == 8  # check for 8 unique elements because i screw these up a lot

def play_game(board: collections.defaultdict[tuple[int, int], int]):
  max_x = max(k[0] for k in board)
  max_y = max(k[1] for k in board)
  for x in range(max_x+1):
    for y in range(max_y+1):
      adj_sum = sum(board[(x+t[0], y+t[1])] for t in adjacency)
      if adj_sum not in board.values() and adj_sum == max(board.values()) + 1 and board[(x, y)] == 0:
        board[(x, y)] = adj_sum
  print(board)

def board_to_data(board: collections.defaultdict[tuple[int, int], int]) -> list[list[int]]:
  if any((x<0 or y<0) for x, y in board):
    diagonal_shift = collections.defaultdict(lambda: 0)
    diagonal_shift.update({(x+1, y+1): value for (x,y), value in board.items()})
    board = diagonal_shift
  max_x = max(k[0] for k in board)
  max_y = max(k[1] for k in board)
  data = np.zeros((max_x+1, max_y+1))
  for (x, y), value in board.items():
    data[y, x] = value
  return data

In [18]:
setup_init_state(board=board)
play_game(board=board)
print(board_to_data(board=board))
play_game(board=board)
print(board_to_data(board=board))

defaultdict(<function <lambda> at 0x7fd7272208c0>, {(0, 0): 1, (2, 2): 1, (-1, -1): 0, (0, -1): 0, (1, -1): 0, (-1, 0): 0, (1, 0): 0, (0, 1): 0, (1, 1): 2, (-1, 1): 0, (0, 2): 0, (1, 2): 3, (-1, 2): 0, (0, 3): 0, (1, 3): 0, (2, -1): 0, (2, 0): 0, (2, 1): 0, (2, 3): 0, (3, -1): 0, (3, 0): 0, (3, 1): 0, (3, 2): 0, (3, 3): 0})
[[0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 2. 0. 0.]
 [0. 0. 3. 1. 0.]
 [0. 0. 0. 0. 0.]]
defaultdict(<function <lambda> at 0x7fd7272208c0>, {(0, 0): 1, (2, 2): 1, (-1, -1): 0, (0, -1): 0, (1, -1): 0, (-1, 0): 0, (1, 0): 0, (0, 1): 0, (1, 1): 2, (-1, 1): 0, (0, 2): 0, (1, 2): 3, (-1, 2): 0, (0, 3): 0, (1, 3): 4, (2, -1): 0, (2, 0): 0, (2, 1): 0, (2, 3): 0, (3, -1): 0, (3, 0): 0, (3, 1): 0, (3, 2): 0, (3, 3): 0, (-1, 3): 0, (0, 4): 0, (1, 4): 0, (2, 4): 0, (3, 4): 0, (4, -1): 0, (4, 0): 0, (4, 1): 0, (4, 2): 0, (4, 3): 0, (4, 4): 0})
[[0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 2. 0. 0. 0.]
 [0. 0. 3. 1. 0. 0.]
 [0. 0. 4. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]


## Packaging some of this functionality

In [42]:
class NumbersAndStones:
  """ Useful namespace for containing the game and its board """
  def __init__(self, initial_state: int = 2):
    self.board: collections.defaultdict[tuple[int, int], int] = collections.defaultdict(lambda: 0)
    self.last_state: collections.defaultdict[tuple[int, int], int] = collections.defaultdict(lambda: 0)
    setup_init_state(board=self.board, init_stones=initial_state)
    self.last_state.update(self.board)
    self.sequence: set[int] = set()
  
  def play_game(self):
    self.last_state = collections.defaultdict(lambda: 0)
    self.last_state.update(self.board)
    play_game(board=self.board)
  
  def evaluate_growth(self) -> bool:
    """ 
    return whether or not growth stagnated (mathematically this <i>must</i> happen)
    irrespective of init state
    True: growth
    False: stagnated
    """
    return not sorted(v for v in self.board.values() if v != 0) == sorted(v for v in self.last_state.values() if v != 0)
  
  def play_sequence(self):
    self.play_game()
    while self.evaluate_growth():
      self.play_game()
      self.sequence.update({*self.board.values()})
  
  @property
  def ndarray(self):
    return board_to_data(board=self.board)

In [43]:
game = NumbersAndStones()
game.play_sequence()
print(game.ndarray)
print(game.sequence)

defaultdict(<function NumbersAndStones.__init__.<locals>.<lambda> at 0x7fd72541f0e0>, {(0, 0): 1, (2, 2): 1, (-1, -1): 0, (0, -1): 0, (1, -1): 0, (-1, 0): 0, (1, 0): 0, (0, 1): 0, (1, 1): 2, (-1, 1): 0, (0, 2): 0, (1, 2): 3, (-1, 2): 0, (0, 3): 0, (1, 3): 0, (2, -1): 0, (2, 0): 0, (2, 1): 0, (2, 3): 0, (3, -1): 0, (3, 0): 0, (3, 1): 0, (3, 2): 0, (3, 3): 0})
defaultdict(<function NumbersAndStones.__init__.<locals>.<lambda> at 0x7fd72541f0e0>, {(0, 0): 1, (2, 2): 1, (-1, -1): 0, (0, -1): 0, (1, -1): 0, (-1, 0): 0, (1, 0): 0, (0, 1): 0, (1, 1): 2, (-1, 1): 0, (0, 2): 0, (1, 2): 3, (-1, 2): 0, (0, 3): 0, (1, 3): 4, (2, -1): 0, (2, 0): 0, (2, 1): 0, (2, 3): 0, (3, -1): 0, (3, 0): 0, (3, 1): 0, (3, 2): 0, (3, 3): 0, (-1, 3): 0, (0, 4): 0, (1, 4): 0, (2, 4): 0, (3, 4): 0, (4, -1): 0, (4, 0): 0, (4, 1): 0, (4, 2): 0, (4, 3): 0, (4, 4): 0})
defaultdict(<function NumbersAndStones.__init__.<locals>.<lambda> at 0x7fd72541f0e0>, {(0, 0): 1, (2, 2): 1, (-1, -1): 0, (0, -1): 0, (1, -1): 0, (-1, 0): 